In [1]:
# This code aims at retreiving financial indicators of hermès to create the income statement

import yfinance as yf

ticker = yf.Ticker('RMS.PA')  # Hermès ticker on Euronext Paris
income_stmt = ticker.financials
print(income_stmt)

                                                       2024-12-31  \
Tax Effect Of Unusual Items                                   0.0   
Tax Rate For Calcs                                       0.286847   
Normalized EBITDA                                    6992000000.0   
Total Unusual Items                                           NaN   
Total Unusual Items Excluding Goodwill                        NaN   
Net Income From Continuing Operation Net Minori...   4603000000.0   
Reconciled Depreciation                               844000000.0   
Reconciled Cost Of Revenue                           4511000000.0   
EBITDA                                               6992000000.0   
EBIT                                                 6148000000.0   
Net Interest Income                                   283000000.0   
Interest Expense                                              NaN   
Interest Income                                       283000000.0   
Normalized Income                 

In [17]:
indicators = {
    "Total Revenue": "Net Sales (Revenue)",
    "Cost Of Revenue": "Cost of Goods Sold (COGS)",
    "Gross Profit": "Gross Profit",
    "General And Administrative Expense": "Selling, General & Administrative Expenses (SG&A)",
    "Operating Income": "Operating Income",
    "Interest Expense": "Interest Expense",
    "Pretax Income": "Income Before Taxes (EBT)",
    "Tax Provision": "Income Tax Expense",
    "Net Income": "Net Income"
}

# Extract relevant data
df_income = income_stmt.loc[list(indicators.keys())]

# Rename index for readability
df_income.index = [indicators[item] for item in df_income.index]

# Transpose the data for better formatting
df_income = df_income.T

# Identify the latest and previous year dynamically
latest_year = df_income.index[0]  # Most recent year available
previous_year = df_income.index[1]  # Previous year

# Keep only the latest year data
df_income_filtered = df_income.loc[[latest_year]].copy()

# Calculate the percentage growth for each indicator
df_income_filtered = df_income_filtered.T  # Transpose to calculate per indicator
df_income_filtered["% Growth"] = ((df_income_filtered[latest_year] - df_income.loc[previous_year]) / df_income.loc[previous_year]) * 100
df_income_filtered = df_income_filtered.round(2)  # Round values for clarity

# Display results
print(df_income_filtered)

                                                  2024-12-31 00:00:00  \
Net Sales (Revenue)                                     15170000000.0   
Cost of Goods Sold (COGS)                                4511000000.0   
Gross Profit                                            10659000000.0   
Selling, General & Administrative Expenses (SG&A)        3569000000.0   
Operating Income                                         6148000000.0   
Interest Expense                                                  NaN   
Income Before Taxes (EBT)                                6432000000.0   
Income Tax Expense                                       1845000000.0   
Net Income                                               4603000000.0   

                                                    % Growth  
Net Sales (Revenue)                                12.981306  
Cost of Goods Sold (COGS)                          21.263441  
Gross Profit                                        9.807356  
Selling, General 

In [ ]:
# The following script allows to visualise different scenarii for the evolution of Hermès's revenue.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Historical Revenue Data (in billions of euros, example values)
historical_revenue = {
    2019: 6.88,
    2020: 6.39,
    2021: 8.98,
    2022: 11.60,
    2023: 13.42  # Estimated
}

# Growth rate assumptions
growth_rates = {
    "Pessimistic": 0.04,  # 4% annual growth
    "Neutral": 0.07,      # 7% annual growth
    "Optimistic": 0.12    # 12% annual growth
}

# Forecast period
years_forecast = list(range(2024, 2029))

# Creating DataFrame
df = pd.DataFrame({"Year": list(historical_revenue.keys()) + years_forecast})

# Adding historical revenue
df["Revenue"] = df["Year"].map(historical_revenue)

# Generate projections
last_revenue = historical_revenue[2023]
for scenario, rate in growth_rates.items():
    forecast = []
    revenue = last_revenue
    for year in years_forecast:
        revenue *= (1 + rate)  # Apply growth rate
        forecast.append(revenue)
    df[scenario] = [None] * len(historical_revenue) + forecast

# Display DataFrame
import ace_tools as tools
tools.display_dataframe_to_user(name="Hermès Financial Projections", dataframe=df)

# Plot the projections
plt.figure(figsize=(10, 5))
plt.plot(df["Year"], df["Revenue"], marker='o', label="Historical Revenue", linestyle='dashed')
plt.plot(df["Year"], df["Pessimistic"], marker='o', label="Pessimistic Projection")
plt.plot(df["Year"], df["Neutral"], marker='o', label="Neutral Projection")
plt.plot(df["Year"], df["Optimistic"], marker='o', label="Optimistic Projection")

plt.xlabel("Year")
plt.ylabel("Revenue (Billion €)")
plt.title("Hermès Revenue Forecast (Pessimistic, Neutral, Optimistic)")
plt.legend()
plt.grid(True)
plt.show()
